In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

df = pd.read_csv('dataset.csv')

In [2]:
# Define the pairs of columns to plot
columns_pairs = [
    ('tri_usdt_oracle_new', 'tri_usdt_ref'),
    ('tri_usdc_oracle_new', 'tri_usdc_ref'),
    ('tri_llama_oracle_new', 'tri_llama_ref'),
    ('tri_lsd_oracle_new', 'tri_lsd_ref'),
    ('tri_crv_oracle_new', 'tri_crv_ref'),
]

In [3]:
# Loop over each pair and create two plots for each
for oracle_col, ref_col in columns_pairs:
    # Extract the currency code from the column name for titles
    currency_code = oracle_col.split('_')[1]  # Extract the currency code from the column name
    
    # Graph 1: Oracle vs. Ref comparison
    fig1 = go.Figure()
    
    # Adding the oracle data as markers
    fig1.add_trace(
        go.Scatter(x=df[ref_col], y=df[oracle_col], name=f'{oracle_col}', mode='markers')
    )
    
    # Adding the perfect match line without displaying points
    fig1.add_trace(
        go.Scatter(x=df[ref_col], y=df[ref_col], name='Match', mode='lines')
    )
    
    # Update layout for the first graph
    fig1.update_layout(title=f'{currency_code.upper()} Oracle Performance Analysis', xaxis_title=ref_col, yaxis_title=oracle_col)
    
    # Save the figure
    image_path1 = f'images/{currency_code.upper()}_Oracle_vs_Ref.png'
    fig1.write_image(image_path1)
    
    # Graph 2: Oracle_new and Ref over Time
    fig2 = go.Figure()
    
    # Adding the oracle_new data
    fig2.add_trace(
        go.Scatter(x=df['Timestamp'], y=df[oracle_col], name=f'{oracle_col}', mode='lines+markers')
    )
    
    # Adding the ref data
    fig2.add_trace(
        go.Scatter(x=df['Timestamp'], y=df[ref_col], name=f'{ref_col}', mode='lines+markers')
    )
    
    # Update layout for the second graph
    fig2.update_layout(title=f'{currency_code.upper()} Oracle vs. Ref over Time', xaxis_title='Time', yaxis_title='Value')
    
    # Save the figure
    image_path2 = f'images/{currency_code.upper()}_Oracle_vs_Ref_Over_Time.png'
    fig2.write_image(image_path2)

In [5]:
# Prepare data for plotting
currency_codes = []
mean_variances = []
median_variances = []
percentile_95_variances = []
percentile_99_variances = []

for oracle_col, ref_col in columns_pairs:
    # Calculate variance in percent
    df['variance_percent'] = ((df[oracle_col] - df[ref_col]) / df[ref_col]) * 100
    
    # Calculate statistics
    mean_variance_percent = df['variance_percent'].mean()
    median_variance_percent = df['variance_percent'].median()
    percentile_95_variance_percent = df['variance_percent'].quantile(0.95)
    percentile_99_variance_percent = df['variance_percent'].quantile(0.99)
    
    # Extract the currency code
    currency_code = oracle_col.split('_')[0] + "-" + oracle_col.split('_')[1]
    currency_codes.append(currency_code.upper())
    mean_variances.append(mean_variance_percent)
    median_variances.append(median_variance_percent)
    percentile_95_variances.append(percentile_95_variance_percent)
    percentile_99_variances.append(percentile_99_variance_percent)

# Create a Plotly figure
fig = go.Figure()

# Define the positions of the bars
index = np.arange(len(columns_pairs))
bar_width = 0.2

# Adding bars for each statistic
fig.add_trace(go.Bar(x=[x - 1.5*bar_width for x in index], y=mean_variances, name='Mean Variance', marker_color='skyblue'))
fig.add_trace(go.Bar(x=[x - 0.5*bar_width for x in index], y=median_variances, name='Median Variance', marker_color='lightgreen'))
fig.add_trace(go.Bar(x=[x + 0.5*bar_width for x in index], y=percentile_95_variances, name='95th Percentile Variance', marker_color='orange'))
fig.add_trace(go.Bar(x=[x + 1.5*bar_width for x in index], y=percentile_99_variances, name='99th Percentile Variance', marker_color='salmon'))

# Update the layout
fig.update_layout(
    title='Mean, Median, 95th, and 99th Percentile Variance for Each Pair',
    xaxis=dict(title='Currency Pair', tickmode='array', tickvals=index, ticktext=currency_codes),
    yaxis=dict(title='Variance (%)'),
    barmode='group'
)

fig.show()
fig.write_image('images/stats.png')